In [1]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
import re
import pandas as pd
import time 
from tqdm.notebook import tqdm

In [2]:
import matplotlib.pyplot as plt
plt.rc('font', family = 'NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

In [3]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [4]:
chrome_options = webdriver.ChromeOptions()

In [5]:
url = 'https://www.opinet.co.kr/searRgSelect.do'
driver = webdriver.Chrome(service=Service('../driver/chromedriver'), options = chrome_options)
driver.get(url)

In [6]:
driver.get(url) ## 싼 주유소 찾기 url 들어가기

In [7]:
si = driver.find_element(By.ID, 'SIDO_NM0')
si.send_keys('서울특별시') 
## 서울 지역 선택하기 

# 왜 몇몇 구에서 오류가 나는지 이유를 모르겠음..

In [46]:
def on_off(gif):
    gif = gif.get_attribute('src')
    on_off = re.search('(off)?.gif', gif).group()

    if len(on_off) == 4 :
        on_off = 'ON'
    else:
        on_off = 'OFF'
    
    return on_off


gu = driver.find_element(By.ID, 'SIGUNGU_NM0')
gu_list = gu.find_elements(By.TAG_NAME, "option")

gu_names = [option.get_attribute('value')for option in gu_list]
gu_names.remove("")
gu_names.remove('성북구')
gu_names.remove('동대문구')
gu_names.remove('영등포구')
gu_names.remove('종로구')
gu_names.remove('중구')


oil_information = []

for gu_name in tqdm(gu_names):
    gu = driver.find_element(By.ID, 'SIGUNGU_NM0')
    gu.send_keys(gu_name)
    driver.implicitly_wait(time_to_wait=2)
    
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')

    content = soup.find('tbody', id='body1')
    contents = content.find_all('td', class_='rlist')


    oil_path = []

    for i in range(len(contents)):
        oil_path.append(f'''//*[@id="body1"]/tr[{i+1}]/td[1]''')
    
    
    for n in range(len(contents)):
        driver.implicitly_wait(time_to_wait=3)
        search_docs = driver.find_element(By.XPATH, oil_path[n]).click()
    
        leq = driver.page_source
        soup = BeautifulSoup(leq, 'html.parser')
        
        
        name = soup.find('label', id ='os_nm').text
        mark = soup.find('label' , id='poll_div_nm').text
        addr = soup.find('label', id='rd_addr').text
        m_gasol = soup.find('label', id='b027_p').text
        oil = soup.find('label', id='d047_p').text
 
        wash = driver.find_element(By.ID, 'cwsh_yn')
        wash = on_off(wash)
    
        charge = driver.find_element(By.ID, 'lpg_yn')
        charge = on_off(charge)
    
        maint = driver.find_element(By.ID, 'maint_yn')
        maint = on_off(maint)
    
        store = driver.find_element(By.ID, 'cvs_yn')
        store = on_off(store)
    
        sel = driver.find_element(By.ID, 'sel24_yn')
        sel = on_off(sel)
    
        each = {'매장이름' : name,
                '상표' : mark,
                '주소' : addr,
                '휘발유' : m_gasol,
                '경유' : oil,
                '세차장' : wash,
                '충전소' : charge,
                '경정비' : maint,
                '편의점' : store,
                '24시' : sel}
        oil_information.append(each)
        
df_oil = pd.DataFrame(oil_information)        


  0%|          | 0/20 [00:00<?, ?it/s]

In [47]:
df_oil

,매장이름,상표,주소,휘발유,경유,세차장,충전소,경정비,편의점,24시
0,(주)보성 세곡주유소,SK에너지,서울 강남구 헌릉로 731 (세곡동),"1,663","1,839",ON,OFF,OFF,OFF,OFF
1,방죽주유소,GS칼텍스,서울 강남구 밤고개로 215 (율현동),"1,687","1,899",ON,OFF,OFF,OFF,OFF
2,현대오일뱅크(주)직영 산성셀프주유소,현대오일뱅크,서울 강남구 헌릉로 730,"1,689","1,879",ON,OFF,OFF,OFF,OFF
3,자곡셀프주유소,SK에너지,서울 강남구 밤고개로 120 (자곡동),"1,727","1,898",ON,OFF,OFF,OFF,OFF
4,대교주유소,SK에너지,서울 강남구 강남대로 640 (신사동),"1,730","1,910",OFF,OFF,OFF,OFF,OFF
...,...,...,...,...,...,...,...,...,...,...
352,(주)기지에너지,S-OIL,서울 중랑구 용마산로 716 (신내동),"1,645","1,815",OFF,OFF,OFF,OFF,OFF
353,신내주유소,SK에너지,서울 중랑구 용마산로 705 (신내동),"1,658","1,825",ON,OFF,OFF,OFF,OFF
354,범아주유소,S-OIL,서울 중랑구 동일로 881 (묵동),"1,689","1,849",ON,OFF,ON,OFF,OFF
355,신일셀프주유소,SK에너지,서울 중랑구 상봉로 58 (망우동),"1,698","1,899",ON,OFF,ON,OFF,ON


In [ ]:
df_oil.to_csv('df_oil.csv')